In [35]:
from datasets import load_dataset
c4_su = load_dataset("allenai/c4", "su")
c4_su_dev_dict = c4_su.data['validation'].to_pydict()

In [49]:
import pandas as pd
mabl_su_df = pd.read_csv('../langdata/su.csv')
# mabl_su_df = pd.read_csv('../translate-test/su.csv')
# mabl_su_df = pd.read_csv('../data/syntax_chunked/syntax_tagged_su.csv')

In [50]:
mabl_su_df

,ending1,ending2,startphrase,labels
0,hirup kawas kakekang,hirup kawas bebas.,hirup kawas kurung batok,0
1,hirup kawas kakekang,hirup kawas bebas.,hirup kawas murag bulu bitis.,1
2,jadi jalma teh kudu rajin,jadi jalma teh kudu kedul.,jadi jalma teh kudu hampang birit,0
3,jadi jalma teh kudu rajin,jadi jalma teh kudu kedul.,jadi jalma teh kudu beurat birit.,1
4,mamat lumpat siga gancang,mamat lumpat siga laun.,Mamat lumpatna siga uncal,0
...,...,...,...,...
595,Balik sareng mantan mah seru,Balik sareng mantan mah bosen.,Balik deui sareng mantan sapertos nganggo baju...,1
596,Hubungan jarak jauh pinuh ku rintangan,Hubungan jarak jauh teu aya rintanganana.,Hubungan jarak jauh ibarat arum jeram di sungai,0
597,Hubungan jarak jauh pinuh ku rintangan,Hubungan jarak jauh teu aya rintanganana.,Hubungan jarak jauh ibarat numpakan kano di ta...,1
598,Panghina nyieun leuwih getol,Panghina nyieun nelangsa.,Panghina ti batur mah siga kusir keur mecut kuda,0


In [56]:
start_phrases = mabl_su_df['ending1'].tolist()


In [54]:
c4_su_toy = c4_su_dev_dict['text']

In [55]:
c4_su_dev_dict['text'][:5]

['Etas luctus - Linda Naor\nEtas luctus2014-08-212014-08-21https://lindanaor.com/wp-content/uploads/2019/01/linda-naor-logo.pngLinda Naorhttps://lindanaor.com/wp-content/uploads/2019/01/linda-naor-logo.png200px200px',
 'Lukas 9 (MANGGARAI)\nMori Yésus Wuat Campulu Suad Rasuln (Mat 10:5-15; Mr 6:7-13)\n1Itu kali Mori Yésus bénta campulu suad rasul, poli hitu téing mberes agu kuasa oné isé, kudut wur jing, agu latang te ina taung sanggéd beti.\n2Agu hia wuat isé kudut keréba Adak de Mori Keraéng, agu pandé ina ata,\n3mai taé Diha ngong isé: “Néka ba apa-apa du ngo lako, néka ba do’ar ko bokong, roti ko doi, ko suan baju.\n4Méu émé poli tiba lata oné ca mbaru, ka’éng nitum dengkir méu angkak nitu mais.\n5Émé manga ata situt toé gorid te tiba méu, pé’ang oné mai béndar disés, agu péntang kebok wa’is, cama ného te langi isé.”\n6Itu kali ngod isé, agu isé lako labok taung béo, cang agu wéro keréba di’ad agu pandé ina ata beti néténg tana.\nHi Hérodés Agu Mori Yésus (Mat 14:1-12; Mr 6:14-29)\

### matching

In [170]:
import numpy as np
# based on langchain

def ngram_overlap_score(source: str, example: list[str]) -> float:
    from nltk.translate.bleu_score import (
        SmoothingFunction,  # type: ignore
        sentence_bleu,
    )

    hypotheses = source.split()
    references = [s.split() for s in example]

    return float(
        sentence_bleu(
            references,
            hypotheses,
            smoothing_function=SmoothingFunction().method1,
            auto_reweigh=True,
        )
    )
 
def select_examples(queries: list[str], example_pool: list[str], threshold = -1.0) -> list[dict]:
    selected_examples = []
    queries_size = len(queries)
    pool_size = len(example_pool)
    scores = [0.0] * pool_size

    for i in range(queries_size):
        for j in range(pool_size):
            scores[j] += ngram_overlap_score(
                queries[i], [example_pool[j]]
            )

    score_scratch = [s for s in scores]
    while True:
        arg_max = np.argmax(score_scratch)
        if (score_scratch[arg_max] < threshold) or (score_scratch[arg_max] == 0):
            break

        selected_examples.append((score_scratch[arg_max], example_pool[arg_max]))
        score_scratch[arg_max] = threshold - 1.0

    return scores, selected_examples

scores, selected_examples = select_examples(
    queries = start_phrases[:100], 
    example_pool = c4_su_dev_dict['text'][:100000],
    threshold=0
)

for e in selected_examples:
    print(e)

(0.020436578760388388, 'Azbabun Nuzul. | As-Sholihina\nCategories: (INA), Diary\t| Tags: blog, hidup, kehidupan, menulis, sharing, tujuan hidup')
(0.011897489988714411, '(MP) Long Weekend di Bandung | pinkuonna\n← (MP) Jalan-jalan di Malaysia & Singapura (Part 3)\n(MP) Pembunuhan….Tikus-tikus…:) →')
(0.01127809002315617, "CIKGU ROHANI M.U: Guru menang Anugerah Apicta Terbaik Perdana Menteri\nGuru menang\nAnugerah Apicta Terbaik\np/s : Ini blog Apis\nLife Through My Eyes: A teacher's little steps towards perfection\nAuthor: Muhamad Hafiz Ismail")
(0.007750489665233287, 'Jual Sapi Perah Komplek Perumahan Melong Green Garden Bandung | Jual|Harga|Sapi Qurban|Domba Qurban\njual sapi perah komplek Perumahan Melong Green garden bandung')
(0.006586739964308081, 'Pilari nu bakal kasampak dinten sateuacan bade ranjang? masturbasi normal mawa émosi poto nu leuwih jelas lamun pilari tits kartun porno. A-rupa penah jeung pangaweruh kacamatan erogenous dijamin hasilna orgasms jurig. tits péngkolan B